# Домашнее задание к вебинару 3

In [1]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-06-07 10:39:37--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-06-07 10:39:38--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc082db6bccbded4b854317c7949.dl.dropboxusercontent.com/cd/0/inline/BmtgFeh8-O2PSo_7tzAWV2l-NDHOJ6mIqb0AQYCm9W4pwIHUcewbGgG524gSWhB7xcK9j-uAzvZ1_aMPVyQe9SZlCQibiBWBS_CP6v8zW-2cD3gQF-hAOFzmfFP4fVBmsSvm9vUpfJbQ-Ecm5UjMqc9ye_ROdeVAMlEIS1HFbL194A/file# [following]
--2022-06-07 10:39:38--  https://uc082db6bccbded4b854317c7949.dl.dropboxusercontent.com/cd/0/inline/BmtgFeh8-O2PSo_7tzAWV2l-NDHOJ6mIqb0AQYCm9W4pwIHUcewbGgG524gSWhB7xcK9

Задание:
1. объединить в одну выборку
2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов
(на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)
3. Проверить насколько хорошо работают подходы

In [3]:
# pip install pymorphy2

In [2]:
# pip install stop_words

In [1]:
# pip install annoy

In [5]:
import pandas as pd
import gensim.downloader as api
from gensim.models import Word2Vec, FastText
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string
from sklearn.model_selection import train_test_split
import annoy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import gensim.models
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Embedding
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [6]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['tweet'])
positive

,tweet
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...
...,...
114906,"Спала в родительском доме, на своей кровати......"
114907,RT @jebesilofyt: Эх... Мы немного решили сокра...
114908,"Что происходит со мной, когда в эфире #proacti..."
114909,"""Любимая,я подарю тебе эту звезду..."" Имя како..."


In [7]:
# Загрузка данных и объединение в общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive.head(2)

,text
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."


In [8]:
positive['label'] = 'positive'

In [9]:
positive.head(2)

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive


In [10]:
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = 'negative'
negative.head(2)

,text,label
0,на работе был полный пиддес :| и так каждое за...,negative
1,"Коллеги сидят рубятся в Urban terror, а я из-з...",negative


In [11]:
# объединим датасеты:
df = positive.append(negative)

In [12]:
# разделим на train и test

X_train, X_test, y_train, y_test = train_test_split(df.text, df.label)

Предобработка текста и подготовка к обучению

In [15]:
import nltk
nltk.download('stopwords')

import nltk
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
morpher = MorphAnalyzer()
sw = get_stop_words("ru") + stopwords.words('russian')
punkt = list(string.punctuation)
noise = set(sw + punkt)

In [17]:
#Функция предобработки текста

def preprocess_txt(text):
    return [t for t in [morpher.parse(token)[0].normal_form 
                                for token in word_tokenize(text)] if t not in noise]

In [18]:
#пример обработки твита
preprocess_txt(df.text.iloc[0])

['first_timee',
 'школотый',
 'поверь',
 'самый',
 'd',
 'общество',
 'профилировать',
 'предмет',
 'тип']

In [19]:
#2. Процедура подготовки данных к обучению (создание списка токенов для каждого твита)
assert True

# Preprocess for models fitting
tweet_tokens = []
c = 0
for text in X_train:
    tweet_prep = preprocess_txt(text)
    tweet_tokens.append(tweet_prep)
    c += 1
    
    if c > 100000:
        break

In [20]:
# убираем из токенов X_train'a те, короче 2 символов
tweet_tokens = [i for i in tweet_tokens if len(i) > 2]
tweet_tokens[0]

['rt', 'natacitrus', 'gerard_way77', 'дурень', 'd', 'свой', 'сердце', 'болеть']

In [21]:
# Функция поиска 5 ближайших твитов
def get_response(tweet, index, model, index_map):
    tweet_token = preprocess_txt(tweet)
    vector = np.zeros(300)
    norm = 0
    for word in tweet_token:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    similar_tweet = index.get_nns_by_vector(vector, 5, include_distances=True)
    
    result = pd.DataFrame({'tweet': [index_map[i]  for i in similar_tweet[0]], 
                           'distance' : similar_tweet[1]})
    return result

In [22]:
# Процедура поиска ближайших твитов
def get_model_index(model):
    model_index = annoy.AnnoyIndex(300 ,'angular') # поиск ближайших соседей

    index_map = {}
    counter = 0

    for tweet in X_train:
        n_model = 0
        index_map[counter] = tweet
        tweet_prep = preprocess_txt(tweet)
        
        vector_model = np.zeros(300) #вектор твита

        for word in tweet_prep:
            if word in model:
                vector_model += model[word]
                n_model += 1
        if n_model > 0:
            vector_model = vector_model / n_model #нормализуем вектор твита
       
        model_index.add_item(counter, vector_model) #добавляем элемент для индексации
     
        counter += 1

        if counter > 100000:
            break

    model_index.build(10) #из индексов создаем 10 кластеров
    return model_index, index_map

#### Word2Vec

In [23]:
modelW2V = Word2Vec(sentences=tweet_tokens, size = 300, window=5, min_count=1)

In [24]:
w2v_index, index_map = get_model_index(modelW2V.wv)

In [25]:
print(X_test.iloc[0])
get_response(X_test.values[0], w2v_index, modelW2V.wv, index_map)

Заболеть перед Новым годом... Могу, умею, практикую(


,tweet,distance
0,круто. я заболеваю. только я могу заболеть пер...,0.037711
1,А! Еще и зачеты!!! Почти нет ничего нового и н...,0.046964
2,Как же хочется ощутить магию Нового года...но ...,0.053448
3,@lepilkin3 скоро новый год...скучаю по лету по...,0.056646
4,Где же я буду встречать новый год..(((( хнык.....,0.060756


### Fasttext

In [26]:
modelFT = FastText(sentences=tweet_tokens, size=300, min_count=1, window=5, workers=8)

In [27]:
ft_index, index_map = get_model_index(modelFT.wv)

In [28]:
print(X_test.iloc[0])
get_response(X_test.values[0], ft_index, modelFT.wv, index_map)

Заболеть перед Новым годом... Могу, умею, практикую(


,tweet,distance
0,круто. я заболеваю. только я могу заболеть пер...,0.047828
1,Синяк на шеке:( и болит жудко ...новый год удался,0.076370
2,"Как же я себя хреново чувствую, жесть вообще.....",0.077561
3,"Что то я совсем разболелась :(\nЛюбимый, прихо...",0.085624
4,Давно себя так фигово не чувствовала....(,0.087677


Русскоязычная модель ruwikiruscorpora_upos_cbow_300_10_2021

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
modelGLoVE = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/NLP/glove_rusvectores/model.bin', binary=True)  

In [31]:
glove_index, index_map = get_model_index(modelGLoVE)

In [32]:
print(X_test.values[0])
get_response(X_test.values[0], glove_index,modelGLoVE, index_map)

Заболеть перед Новым годом... Могу, умею, практикую(


,tweet,distance
0,"RT @Loorchik: Что-то мечты не сбываются, обидн...",1.414214
1,Проводили))) а у Каревой опять глаза светятся)...,1.414214
2,@ZaxarBorisych из Нарвы мог бы и через Днепр. ...,1.414214
3,"что то мне подсказывает,что надо лечь спать.за...",1.414214
4,10 минут назад должна была выйти на пары... Чу...,1.414214


Embedding слой

In [33]:
text_dataset = tf.data.Dataset.from_tensor_slices([x for l in tweet_tokens for x in l]) #список всех токенов из x_train

In [34]:
# Создание слоя словаря

vectorize_layer = TextVectorization(
    max_tokens=100000,
    output_mode='int',
    output_sequence_length=1)
#создаем словарь
vectorize_layer.adapt(text_dataset)

In [36]:
#создаем модель
model = Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string)) #на вход подается строка
model.add(vectorize_layer)  #получаем тензор (1, 100), содержащий для каждого слова индексы из словаря 
model.add(Embedding(100000, 300, input_length=100, mask_zero=True))

In [37]:
print(X_test.iloc[0])

Заболеть перед Новым годом... Могу, умею, практикую(


In [42]:
model.predict([X_test.values[0]]).squeeze()

array([-2.95383334e-02, -3.42252031e-02, -3.77389044e-03, -4.95881811e-02,
       -3.26779038e-02,  2.03821994e-02, -3.46355513e-03, -7.66699389e-03,
        3.31770666e-02,  6.81697205e-03,  4.78408076e-02, -4.79100235e-02,
       -2.50929724e-02,  2.89527439e-02, -2.53845453e-02, -8.76744837e-03,
        3.25112455e-02,  4.11555804e-02, -3.71403098e-02, -3.42420489e-02,
        1.40436180e-02, -2.76348591e-02, -9.74904373e-03, -3.02096251e-02,
        1.88514851e-02, -2.64329314e-02, -1.12436041e-02, -2.95649525e-02,
       -3.10887937e-02,  3.15589197e-02,  1.70880072e-02,  1.71979330e-02,
       -3.29567939e-02, -5.91925532e-03, -7.85052776e-03, -1.20766088e-03,
        6.94990158e-03,  7.15558603e-03, -3.29658017e-02,  4.46483009e-02,
       -1.52326114e-02, -1.93562750e-02,  3.59576605e-02, -3.02933343e-02,
       -2.78946050e-02,  3.08464058e-02,  1.90481432e-02,  4.44486253e-02,
        3.06270458e-02,  3.80360521e-02,  3.22297923e-02, -2.77092103e-02,
       -2.72413138e-02, -

In [43]:
# Процедура поиска ближайших твитов
emb_index = annoy.AnnoyIndex(300 ,'angular') # поиск ближайших соседей

emb_map = {}
counter = 0

for tweet in X_train:
    emb_map[counter] = tweet
    vector_emb = model.predict([tweet]).squeeze()
    emb_index.add_item(counter, vector_emb) #добавляем элемент для индексации
    counter += 1
        
    if counter > 100000:
        break

emb_index.build(10) #из 10 кластеров

True

In [44]:
def get_response_emb(model, emb_index, tweet, n_similar_tweets):
    vector = model.predict([tweet])[0][0]

    similar_tweet = emb_index.get_nns_by_vector(vector, n_similar_tweets, include_distances=True)
    
    result = pd.DataFrame({'tweet': [emb_map[i]  for i in similar_tweet[0]], 
                           'distance' : similar_tweet[1]
                          })
    return result

In [45]:
print(X_test.values[0])
get_response_emb(model, emb_index, X_test.values[0], 5)

Заболеть перед Новым годом... Могу, умею, практикую(


,tweet,distance
0,@DarvinOfficial пиздец:( посмотрела его обраще...,0.0
1,"@irina_meat, ты почему вк не заходишь?;( споко...",0.0
2,@odnakoyak аахаххааха блин я знала. я зналаааа...,0.0
3,"Я умоляю вернись,но понемаю что тебе не как;(с...",0.0
4,Застряла в candy crush на одном уровне и уже р...,0.0


**Выводы:**  
Наилучшйи результат показала модель Fasttext (нашла наиболее близкие твиты)  
К сожалению, не удалось разобраться с русскоязычной моделью ruwikiruscorpora_upos_cbow_300_10_2021 не удалось полностью разобраться, результат ее работы кажется странным, как и результат работы нейронной сети со слоем Embedding
